<a href="https://colab.research.google.com/github/mburkasov/Financial-forecasting/blob/main/Case_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib as plt
import pandas as pd
import statsmodels.api as sm

from math import sqrt
from google.colab import drive
from sklearn import preprocessing
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
drive.mount('/content/drive')
df1 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/job_satisfaction_train.xlsx')

In [ ]:
df = df1.copy()
#замена текстовых значений
for column in df.select_dtypes(include='object'):
  if len(df[column].value_counts().tolist()) < 10:
    for n, title in enumerate(df[column].value_counts().index.tolist()):
      df[column] = df[column].replace(title, n)
#отсортировать по макс значению
df = df.loc[:, df.max().sort_values(ascending=False).index]
#исправить странную фигню где интернет 8 и 9??
df['internet'][df['internet']>1] = 1
#переставить столбик
jobsat = np.array(df.pop('jobsat').dropna().astype('int64'))
print(jobsat)
df.insert(0,'jobsat',jobsat)
#колонки 
predict = df.iloc[5400:,1:]
df = df.dropna()
#сделать категориями
#for column in df:
#  if df[column].max() < 10:
#    df[column] = df[column].astype('category')

MinMaxScaler(copy=True, feature_range=array([5, 4, 3, ..., 4, 3, 1]))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#функция вифа
def getvif(X):
  vif = pd.DataFrame()
  vif["variables"] = X.columns
  X = np.array(X, dtype = float)
  vif["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
  vif = vif.sort_values(by=['VIF'], ascending=False)
  return vif
#prediction and export
def export_prediction(model, x, columns = x.columns, filename = 'output'):
  pred = model.predict(sm.add_constant(x[columns]))
  export=pd.concat([pd.DataFrame(pred),pd.DataFrame(x)],axis=1)
  export.to_excel(f'/content/drive/My Drive/Colab Notebooks/{filename}.xlsx')
#forward selection
def forward_selection(data, target):
    all_columns = data.columns.tolist()
    model_columns = []
    maxf = 0
    while (len(all_columns)>0):
      all_columns = list(set(all_columns)-set(model_columns))
      currmaxf = 0
      for column in all_columns:
        model = sm.OLS(target, sm.add_constant(data[model_columns+[column]])).fit()
        new_f = model.fvalue
        if (currmaxf < new_f):
          currmaxf = new_f
          best_column = column

      if (maxf < currmaxf):
        maxf = currmaxf

        model_columns.append(best_column)
      else:
        break
    return model_columns
#backwards elimination
def backward_elimination(data, target, crit = 0.05):
    all_columns = data.columns.tolist()
    bad_columns = []
    while (len(all_columns)>1):
      all_columns = list(set(all_columns)-set(bad_columns))
      model = sm.OLS(target, sm.add_constant(data[all_columns])).fit()
      pval = model.pvalues.sort_values(ascending=False)
      if pval[0]>crit:
        bad_columns.append(pval.index[0])
      else:
        break
    return all_columns
#standart error of the model
def model_error (data, columns, step = 600, type = 'OLS'):
  data = data[['jobsat']+columns].dropna()
  predict = []
  for i in range(0,len(data.index),step):
    train = data.iloc[np.r_[0:i, i+step:5400],:]
    pred = data.iloc[i:i+step,1:]
    y = train.iloc[:,0]
    x = train.iloc[:,1:]
    x = sm.add_constant(x)
    pred = sm.add_constant(pred)
    model = getattr(sm,type)(y,x).fit()
    predict += list(model.predict(pred))
  data['prediction'] = predict
  data['sterr'] = (data['jobsat'] - data['prediction']) ** 2
  sterr = sqrt(data['sterr'].sum() / (len(data.index) - 1 - len(columns)))
  print('St. err:', sterr)
  return(sterr)

In [ ]:
df

In [ ]:
#стандартизация данных
x = df#.iloc[:,1:6]
#x_scaled = preprocessing.scale(x)
x_scaled = preprocessing.MinMaxScaler(x)
print
x_scaled = pd.DataFrame(x_scaled, columns=x.columns)
df[x.columns] = x_scaled
#df.select_dtypes(include='category').astype('int64')
df

In [ ]:
X = df.iloc[0:5400,1:]
#X = sm.add_constant(X)
getvif(X)

In [ ]:
df.corr().round(2)

In [ ]:
#eliminating VIF more than 10
crit = 10
X = df.iloc[0:5400,1:]
vif = getvif(X)
poplist = []
while vif.iloc[0,1]>crit:
  poplist.append(vif.iloc[0,0])
  X.pop(vif.iloc[0,0])
  vif = getvif(X)
print('popped:',poplist)
df = df.drop(poplist, axis=1)

popped: ['owntv', 'inccat', 'owncd', 'empcat', 'age', 'carcat', 'ownvcr', 'car']


In [ ]:
x = df.iloc[0:5400,1:]
y = df.iloc[0:5400,0]
col1 = forward_selection(x,y)
print(col1)
X = df[col1].iloc[0:5400,0:]
X = sm.add_constant(X)
Y = df.iloc[0:5400,0]
model = sm.OLS(Y,X).fit()
print(model.summary())
model_error(df, col1)

['employ']
                            OLS Regression Results                            
Dep. Variable:                 jobsat   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     1673.
Date:                Thu, 01 Oct 2020   Prob (F-statistic):          9.52e-319
Time:                        11:59:35   Log-Likelihood:                -8641.6
No. Observations:                5400   AIC:                         1.729e+04
Df Residuals:                    5398   BIC:                         1.730e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3327      0.024     96.9

1.1995493701216402

In [ ]:
type = 'GLSAR'
x = df.iloc[0:5400,1:]
y = df.iloc[0:5400,0]
col2 = backward_elimination(x,y)
print(col2)
X = df[col2].iloc[0:5400,0:]
X = sm.add_constant(X)
Y = df.iloc[0:5400,0]
model2 = getattr(sm,type)(Y,X).fit()
print(model2.summary())
model_error(df, col2, 600, type)
export_prediction(model2, predict, col2, 'finalpred')

['employ', 'ed', 'age', 'inccat', 'empcat']
                           GLSAR Regression Results                           
Dep. Variable:                 jobsat   R-squared:                       0.264
Model:                          GLSAR   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     386.0
Date:                Sat, 03 Oct 2020   Prob (F-statistic):               0.00
Time:                        07:59:02   Log-Likelihood:                -8542.4
No. Observations:                5399   AIC:                         1.710e+04
Df Residuals:                    5393   BIC:                         1.714e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [ ]:
type = 'Logit'
x = df.iloc[0:5400,1:]
y = df.iloc[0:5400,0]
col2 = backward_elimination(x,y)
print(col2)
X = np.array(df[col2].iloc[0:5400,0:])
X = sm.add_constant(X)
Y = np.array(df.iloc[0:5400,0])

model3 = sm.Logit(Y,X).fit()
print(model2.summary())
model_error(df, col2, 600, type)
#export_prediction(model2, predict, col2, 'finalpred')

      employ  ed  age  inccat  empcat
0         23   1   55       3       3
1         35   1   56       4       3
2          4   3   28       2       1
3          0   4   24       2       1
4          5   2   25       1       2
...      ...  ..  ...     ...     ...
5395       5   4   33       2       2
5396       9   3   49       1       2
5397      11   4   35       3       2
5398       3   3   39       1       1
5399       1   4   24       1       1

[5400 rows x 5 columns]
['employ', 'ed', 'age', 'inccat', 'empcat']


ValueError: ignored

In [ ]:
allcol = list(set(df.columns.tolist()) - {'jobsat'})
print(allcol)
X = df.iloc[0:5400,1:]
X = sm.add_constant(X)
Y = df.iloc[0:5400,0]
model = sm.OLS(Y,X).fit()
print(model.summary())
model_error(df, allcol)

['wireless', 'ownpc', 'gender', 'callid', 'multline', 'ownfax', 'pager', 'internet', 'retire', 'ownpda', 'income', 'ed', 'voice', 'callwait', 'news', 'reside', 'marital', 'response', 'address', 'employ']
                            OLS Regression Results                            
Dep. Variable:                 jobsat   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     86.23
Date:                Thu, 01 Oct 2020   Prob (F-statistic):          9.23e-306
Time:                        12:04:42   Log-Likelihood:                -8619.5
No. Observations:                5400   AIC:                         1.728e+04
Df Residuals:                    5379   BIC:                         1.742e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
      

1.2014566376442997

In [ ]:
#bar charts
dfgraph = df.select_dtypes([np.int, np.float])

for i, col in enumerate(dfgraph.columns):
  plt.pyplot.figure()
  sns.barplot(data = dfgraph, x = 'jobsat',  y = col)

In [ ]:
!pip install mord
# choose models
from sklearn.linear_model import LinearRegression, LogisticRegression
from mord import LogisticAT

# instantiate models
model_linear = LinearRegression()
model_1vR = LogisticRegression(multi_class='ovr',
    class_weight='balanced')
model_multi = LogisticRegression(multi_class='multinomial',
    solver='lbfgs',
    class_weight='balanced')
model_ordinal = LogisticAT(alpha=0)  # alpha parameter set to zero to perform no regularisation

In [ ]:
df2 = df[col2]
df2

,empcat,inccat,ed,employ,age
0,3,3,1,23,55
1,3,4,1,35,56
2,1,2,3,4,28
3,1,2,4,0,24
4,2,1,2,5,25
...,...,...,...,...,...
5395,2,2,4,5,33
5396,2,1,3,9,49
5397,2,3,4,11,35
5398,1,1,3,3,39


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
import numpy as np

# divide df into features matrix and target vector
features = df2  #all except quality
target = df['jobsat']

MAE = make_scorer(mean_absolute_error)
folds = 5

print('Mean absolute error:' )
MAE_linear = cross_val_score(model_linear,
    features,
    target,
    cv=folds,
    scoring=MAE)
print('Logistic regression (one versus rest): ', np.mean(MAE_1vR))
MAE_multi = cross_val_score(model_multi,
    features,
    target,
    cv=folds,
    scoring=MAE)
print('Logistic regression (multinomial): ', np.mean(MAE_multi))
MAE_ordinal = cross_val_score(model_ordinal,
    features,
    target,
    cv=folds,
    scoring=MAE)
print('Ordered logistic regression: ', np.mean(MAE_ordinal))

Mean absolute error:
Logistic regression (one versus rest):  1.140925925925926
Logistic regression (multinomial):  nan
Ordered logistic regression:  nan


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown label type: 'continuous'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown label type: 'continuous'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown label type: 'continuous'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition f